# Genotipador versão 000.01
Inicio do desenvolvimento 2021/09/06
Magalhães, RDM.

In [2]:
#Genotipador a partir dos contigs
from io import StringIO
from Bio import SeqIO
import pandas as pd
from Bio.Seq import Seq
import os
import sys

#Função para alterar o cabeçalho dos arquivos fasta - coloca no nome do arquivo no lugar do que está escrito no cabeçalho -- isso é necessário pois todos os arquivos tem contig_1 como cabeçalho.
def change_fasta_header(outdir_v, indir_v):
    import os
    import re
    
    #create output directory
    output_dir = outdir_v #"newheader"
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)

    #take the path to the data files    
    pwd=os.getcwd()+"/"+indir_v#"/data"
    #print(pwd)


    #print(pwd)
    #for dirname, dirnames, filenames in os.walk(pwd):
        #print (dirname)
        #print(dirnames)
       # print(filenames)

    #Get file list inside of data directory.
    for file in os.listdir(pwd):    
        #if ".empty" in dirnames:
        #    dirnames.remove(".empty")
        #print (file)    
        #print(filefull)

        #Select only .nt* or .fa*
        if (r'.nt') in file or (r'.fa') in file:
            #print(filefull)


            myfastalist=list(SeqIO.parse(pwd+"/"+file,'fasta'))


            for record in myfastalist:
                record.id = re.sub("[.].*","",file)
                #print(record.id)

                with open(outdir_v+"/"+file, "w") as f_v:
                    f_v.write('>'+record.id+'\n'+str(record.seq)+'\n')



##########################################################
#Função para obter uma lista de arquivos e posteriormente alinhar
def align_from_files(dir_input, seq_spike):
    #make a list of files to be align
    ls_f = []
    import glob
    ls_f = glob.glob(dir_input+"/*.*")
    #print(ls_f)

    #insert Spike sequence
    ls_f.insert(0, seq_spike)
    #print(ls_f)

    align_v = align_f(ls_f)
    #print(align_v)
    return(align_v)

#Função para obter alinhamento a partir de uma lista de arquivos.
def align_f(list_v):
    from Bio.Align.Applications import MafftCommandline
    
    file_out1= './output/temp_seqs.fasta'
    filenames = list_v
    with open(file_out1, 'w') as outfile:
        for fname in filenames:
            with open(fname) as infile:
                for line in infile:
                    outfile.write(line)
                
    mafft_cline = MafftCommandline("mafft", input = file_out1)
    stdout,stderr = mafft_cline()

    #print("Aligment: \n" + stdout)
    #print ("\n\n Error: \n" + stderr)

    with open("temp_seq_mafft.aln","w") as hamble:
        hamble.write(stdout)
        
    return(str(stdout))
##############################################################


#Função para obtenção dos id e sequencias descriminados a partir da variável obtida do alinhamento.
def fasta_id_seq_from_align(align_v):
    from io import StringIO
    fasta_io = StringIO(align_v)
    records = SeqIO.parse(fasta_io, "fasta")

    id_v = []
    seq_v = []
    for record in records:
        #print("%s\t%s" % (record.id, record.seq[0:10]))
        id_v.append(record.id)
        seq_v.append(record.seq)
    return(id_v, seq_v)
    #print(id_v)
    #print(seq_v)

#Função para carregar as informações presentes na tabela de mutações
def load_mutation_tab(file_tb_name_v):
    import pandas as pd
    df_fe = pd.read_excel(file_tb_name_v, index_col=0)
    #print(df_fe)
    
    #df to dict
    dict_mut = {}
    for dc_v in range(0,len(df_fe.iloc[:,0])):
        if df_fe.index[dc_v] in dict_mut:
            dict_mut[df_fe.index[dc_v]].append(list(df_fe.iloc[dc_v,]))
        else:
            dict_mut[df_fe.index[dc_v]] = list(df_fe.iloc[dc_v,])
    #df_fe.iloc[dc_v,]
    #df_fe.index[dc_v]
    #print(dict_mut)
    return(dict_mut)
 

#função para inicio da sequencia após alinhamentos
def pos_start_f(seq):
    ct = 0
    
    for char_v in seq:
        if(char_v != "-"):
            #print(ct)
            ct_i = ct
            break
            
        ct += 1
    
    #Caso não tenha nenhum alinhamento e apenas "-"
    try:
        ct_i
    except: 
        ct_i = len(seq)
        
        
    ct = 0
    ct_f = 0
    #print(seq[ct_i:])
    for char_v2 in seq[ct_i:]:
        if(char_v2 == "-"):
            #print(ct)
            ct_f = ct
            break
        ct += 1
    
    #print(str(ct_i)+"\t"+str(ct_f))
    return(ct_i,ct_f)




        
#função para obter a posição em frame a traduzir a sequencia corretamente
def seq_translate_f(seq_v_i):
    #Obter a posição da frame
    
    start_pos,end_range_pos = pos_start_f(seq_v_i)

    if(start_pos > 3):
        start_frame_pos = start_pos + (3-start_pos%3)
        start_aa_pos = (start_frame_pos/3)+1
        if(end_range_pos != 0):
            end_pos = start_pos+end_range_pos
            end_frame_pos = end_pos-(end_pos%3)
            end_aa_pos = (end_frame_pos/3)+1
            #print(len(seq_v_i))
            #print(start_frame_pos)
            #print(end_frame_pos)
            #print(end_aa_pos)

        else:
            end_frame_pos = (len(seq_v_i)-(len(seq_v_i)%3))
            end_aa_pos = (end_frame_pos/3)
            #print(end_aa_pos)
            #print(end_frame_pos)
        #print(start_aa_pos)
        #print(start_frame_pos)
    elif(start_pos == 0):
        start_frame_pos = 0
        start_aa_pos = 1
        if(end_range_pos != 0):
            end_pos = start_pos+end_range_pos
            end_frame_pos = end_pos-(end_pos%3)
            end_aa_pos = (end_frame_pos/3)+1
            #print(end_aa_pos)
            #print(end_frame_pos)
        else:
            end_frame_pos = (len(seq_v_i)-(len(seq_v_i)%3))
            end_aa_pos = (end_frame_pos/3)
            #print(len(seq_v_i))
            #print(start_frame_pos)
            #print(end_frame_pos)
            #print(end_aa_pos)

    else:
        start_frame_pos = 3
        start_aa_pos = 2
        if(end_range_pos != 0):
            end_pos = start_pos+end_range_pos
            end_frame_pos = end_pos-(end_pos%3)
            end_aa_pos = (end_frame_pos/3)+1
            #print(end_aa_pos)
            #print(end_frame_pos)
        else:
            end_frame_pos = (len(seq_v_i)-(len(seq_v_i)%3))
            end_aa_pos = (end_frame_pos/3)


    #Obter a sequencia a partir dessa posição.
    #seq2 = (seq_v_i)[start_frame_pos:]
    #print(seq2)
    #if(end_pos != 0):
    seq2 = (seq_v_i)[start_frame_pos:end_frame_pos]
    #else:
    #    seq2 = (seq_v_i)[start_frame_pos:]

    #print(seq2)
    #print(seq2)
    #print(type(seq2))
    seq2_tr = seq2.translate()
    #print(seq2_tr)
    #print(seq2_tr)
    return(start_aa_pos, end_aa_pos, seq2_tr)

#Função para obter a lista com as mutações encontradas e o dataframe com a descrição
def list_df_make_f(mut_list_PTN, seq2_tr,start_aa_pos, end_aa_pos):
    #Obter a lista e o DataFrame das mutações encontradas
    list_result = []
    #print("aa_pos"+"\t"+"aa_Fd"+"\t"+"aa_WT"+"\t"+"aa_Mut")
    list_coluns_v = ["aa_pos","aa_WT","aa_Mut","aa_Fd","aa_Ch"]
    df_result_v = pd.DataFrame()

    for coord_v in mut_list_PTN:

        #If start of aa sequence was before the first mutation contig
        if(int(start_aa_pos) < coord_v[0] and end_aa_pos > coord_v[0]):
            aa_F = seq2_tr[int(coord_v[0])-int(start_aa_pos)]
            aa_pos = coord_v[0]
            aa_wt = coord_v[1]
            aa_Mut = coord_v[2]


            #print(str(coord_v[0])+"\t"+aa_F+"\t"+coord_v[1]+"\t"+aa_Mut)
            #print(list_temp_v)
            #print(df_temp_v)

            #if a mutation identified... 
            if(aa_F == aa_Mut):
                #List used to check
                list_result.append(1)
                #DataFrame used in results
                list_temp_v = [str(aa_pos),aa_wt,aa_Mut,aa_F,"1"]
                df_temp_v = pd.DataFrame([list_temp_v], columns=list_coluns_v)
                df_result_v = df_result_v.append(df_temp_v,ignore_index=True)


            #If no mutation identified
            else:
                list_result.append(0)

                list_temp_v = [str(aa_pos),aa_wt,aa_Mut,aa_F,"0"]
                df_temp_v = pd.DataFrame([list_temp_v], columns=list_coluns_v)
                df_result_v = df_result_v.append(df_temp_v,ignore_index=True)


        else:  #If contig was shorter than fisrt mutation position

            #aa_F = seq2_tr[int(coord_v[0])-int(start_aa_pos)]
            #aa_Mut = coord_v[2]

            list_result.append(2)

            list_temp_v = [str(coord_v[0]),coord_v[1],coord_v[2],"NA","NA"]
            df_temp_v = pd.DataFrame([list_temp_v], columns=list_coluns_v)
            df_result_v = df_result_v.append(df_temp_v,ignore_index=True)

    #print(list_result)
    #print(df_result_v)
    return(list_result, df_result_v)

#Test to find if there is a match
#Função para correlacionar as listas
def teste_list_f(list_test):
    for key_v in dict_mut_v:
        if list_test == dict_mut_v[key_v]:
            #print(key_v)
            #print(df_result_v)
            return(key_v)
            break

            
def create_dir_or_del_files (folder_v):
    import os, shutil
    
    if not os.path.exists(folder_v):
        os.makedirs(folder_v)
    else:
        folder = folder_v
        for filename in os.listdir(folder):
            file_path = os.path.join(folder, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print('Failed to delete %s. Reason: %s' % (file_path, e))

#main  

dir_initial_data = "data6_Placa20211007"
dir_data = "fasta_temp"

dir_out_v = "Placa_20211020"
out_file_v = 'Genotipagem_20211021.xlsx'
out_dir_file_v = dir_out_v+"/"+out_file_v 

#Cria diretório para fasta ou deleta arquivos no diretório de saída se houver.
create_dir_or_del_files(dir_data)

#Cria diretório de saída se não existir:
if not os.path.exists(dir_out_v):
    os.makedirs(dir_out_v)

#Correção do cabeçalho fasta de contig_1 para o nome do arquivo.
change_fasta_header(dir_data, dir_initial_data)   


#Construção do fasta e alinhamento.
#dir_data = "newheader"
ref_seq_file = "Spike_SeqNucl_LineageB_Wuhan_EPI_ISL_402123_2019.fasta"
align_v = align_from_files(dir_data, ref_seq_file)
#print(align_v)

#Obtendo id e sequencias dos contigs após alinhamento.    
id_v,seq_v = fasta_id_seq_from_align(align_v)

#Carregar as informações presentes na tabela de variantes/mutações
#Caso tenha alteração nessa tabela não esquecer de alterar também a lista de mutações abaixo.
dict_mut_v = load_mutation_tab("Tabela_Variant_Mutations_20210909.xlsx")
#print(dict_mut_v)

#Lista com as mutações
mut_list_PTN = [[346,"R","K"],[417,"K","T"],[417,"K","N"],[452,"L","Q"],[452,"L","R"],[477,"S","N"],[478,"T","K"],[484,"E","K"],[484,"E","Q"],[490,"F","S"],[501,"N","Y"],[614,"D","G"],[655,"H","Y"],[677,"Q","H"],[681,"P","R"],[681,"P","H"],[701,"A","V"]]
#print([i[0] for i in mut_list_PTN[0:]])
#print(len(mut_list_PTN))

#Fazer loop aqui para bustar todos os registros
#Obtendo sequencia de aa

#Carregando arquivo a ser escrito no loop
#writer_f = pd.ExcelWriter('testFile.xlsx', engine='xlsxwriter')

df_summary_out = pd.DataFrame()
dic_out_print = {}

ct_seq_v = 1
for seq_nc_v in seq_v[1:]:
    #print(seq_nc_v)
    st_aa_pos, ed_aa_pos, seq2_tr = seq_translate_f(seq_nc_v)
    print(st_aa_pos)
    print(ed_aa_pos)
    #print(seq2_tr)

    #Obtendo list de mutações e descrição
    ls_temp,df_temp_res = list_df_make_f(mut_list_PTN, seq2_tr, st_aa_pos, ed_aa_pos)
    print(ls_temp)
    print(df_temp_res)

    #Obtendo correlação com lista de variantes
    result_variant_v = teste_list_f(ls_temp)
    print(id_v[ct_seq_v])
    print(str(result_variant_v)+"\n\n")
    
    df_summary_out = pd.concat([df_summary_out,pd.DataFrame({"Sample":[id_v[ct_seq_v]],"Variant Name":[result_variant_v]})])
    #Escrevendo para o arquivo detalhado.
    df_out0 = pd.DataFrame({"aa_pos":["Sample Name:"],"aa_WT":[id_v[ct_seq_v]]})
    df_out1 = pd.DataFrame({"aa_pos":["Variant Name:"],"aa_WT":[result_variant_v]})
    df_out_print = pd.concat([df_temp_res,df_out0,df_out1])
    #dic_out_print[id_v[ct_seq_v]] = [df_out_print,result_variant_v]
    dic_out_print[id_v[ct_seq_v]] = df_out_print
    
    #df_out_print.to_excel(writer_f,sheet_name=(id_v[ct_seq_v])[0:10], index = False)
    #result_variant_v.to_excel(writer_f,sheet_name=(id_v[ct_seq_v])[0:10])
    
    ct_seq_v+=1


#writer_f.save()
#writer_f.close()

#print(df_summary_out)    
#print(dic_out_print)

#Imprimindo arquivo de saida:

writer_out_f = pd.ExcelWriter(out_dir_file_v, engine='xlsxwriter')
df_summary_out.to_excel(writer_out_f,sheet_name="SUMMARY", index = False)
for key_v in dic_out_print:
    #(dic_out_print[key_v])[0].to_excel(writer_out_f,sheet_name=key_v[0:20], startrow = 0, startcol = 0, index = False)
    dic_out_print[key_v].to_excel(writer_out_f,sheet_name=key_v[0:20], index = False)
    
    #worksheet = writer.sheets[]
    #worksheet.write(0, 0, "Id_Sample="+str(key_v))
    #worksheet.write(1, 0, "Variant Name="+str((dic_out_print[key_v])[1]))
writer_out_f.save()
                        
                        



344.0
750.0
[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
   aa_pos aa_WT aa_Mut aa_Fd aa_Ch
0     346     R      K     R     0
1     417     K      T     K     0
2     417     K      N     K     0
3     452     L      Q     R     0
4     452     L      R     R     1
5     477     S      N     S     0
6     478     T      K     K     1
7     484     E      K     E     0
8     484     E      Q     E     0
9     490     F      S     F     0
10    501     N      Y     N     0
11    614     D      G     G     1
12    655     H      Y     H     0
13    677     Q      H     Q     0
14    681     P      R     R     1
15    681     P      H     R     0
16    701     A      V     A     0
FN8863
Delta(B.1.617.2)


342.0
751.0
[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0]
   aa_pos aa_WT aa_Mut aa_Fd aa_Ch
0     346     R      K     R     0
1     417     K      T     K     0
2     417     K      N     K     0
3     452     L      Q     R     0
4     452     L      R     R     1
5     

In [131]:
def del_files_from_folder_f (folder_v):
    import os, shutil
    folder = folder_v
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

del_files_from_folder_f("newheader")

# Alterações para a proxima versão:
- ERRO IDENTIFICADO!!!, caso haja alguma deleção as sequencias seguintes ficam com problema.
- Deletar os diretórios de cabeçalho automaticamente, armazenar os arquivos gerados automaticamente.
- GUI ou alternativa para manipulação mais fácil
- Screening de todas as mutações no contig
- Acréscimo de mais mutações de interesse a serem verificadas também de maneira pontual.
- Apontamento das novas mutações para checagem no espectro... (checagem remota do espectro?)
- Probabilidade de mutações em cada posição para a genotipagem mais precisa??? (entender melhor o que o PANGOLIM faz)